# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "Xwin-LM/Xwin-LM-13B-V0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    # trust_remote_code=True, # ⚠️ Microsoft/Phi-1.5 のみ必要
)

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    #prompt_template = "<s>[INST] <<SYS>>\nあなたは誠実で優秀な日本人のアシスタントです。ユーザは日本語だけを理解するため、あなたは日本語だけを話します。ユーザの質問や指示に回答してください。\n<</SYS>>\n\n{input} [/INST] "
    (
        prompt_template := "好奇心旺盛なユーザとAIアシスタントが会話しています。"
                "アシスタントは、ユーザの質問に対して、親切・詳細で役立つ回答を提供します。 "
                "USER: {input_text} "
                "ASSISTANT:"
    )
    return prompt_template.format(input_text=input_text)

def generate_text(input_text):
    formatted_prompt = format_prompt(input_text)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=4096,
        temperature=0.7,
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            writer.write({"pred": generated_text})

            print("========================================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print()

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A. 1. 目標設定: 明確な仕事の目標を設定し、達成することで熱意を高めることができます。目標を細かく分解し、一日一歩で実現することができるように計画しましょう。
2. 成果主義の強調: 仕事の成果に焦点を当て、成果を測ることで熱意を高めることができます。成果を明確に定義し、自分の成果に対する誇りを持って仕事に取り組むことが重要です。
3. 自己管理: 仕事のバランスを保ち、適切な休憩やリフレッシュを取ることで、熱意を維持することができます。ストレスを軽減し、精神的にリフレッシュされた状態で仕事に取り組むことが大切です。
4. チームワークの強化: チームと協力し、仕事に取り組むことで、熱意を高めることができます。チームメイトとのコミュニケーションを大切にし、相互に励まし合い、仕事を進めていくことが重要です。
5. 成長の意識: 仕事を通じて自分のスキルや知識を向上させることで、熱意を高めることができます。スキルアップや研修などを通じて、自分の成長を感じ、仕事に対する熱意を維持することができます。

Q. クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
A. ある日、森の中で暮らすクマのケンタが、海辺に出くわします。そこには、アザラシたちが泳ぎながら遊んでいるのが確認できます。ケンタは、アザラシたちの楽しい姿に興味を持ち、彼らと友達になることを決心します。

ケンタはアザラシたちと遊び、泳ぎながら楽しい時間を過ごします。アザラシたちはケンタに、海の中の宝物や珊瑚礁の美しさを教えてくれます。一方、ケンタはアザラシたちに、森の中の様々な植物や動物についても教えていきます。

やがて、二度と会えなくなる可能性を感じたケンタは、アザラシたちとの別れを惜しみます。しかし、アザラシたちはケンタに、絶対に会える場所があると告げ、その場所の方向を示します。その場所は、彼らが住んでいる森と海の中間にある、美しい海岸線です。

ケンタはアザラシたちとの別れを惜しみながら、彼らが教えてくれた道をたどり、森を出ます。数日後、ケンタは海岸線に到着し、アザラシたちと再会します。彼らは、今度はケンタに、海で暮らす生活の楽しさや、宝物を探す冒険を教えてくれます。

ケンタはアザラ

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Q. 以下の表から、すべての科目名を抽出してください。
| 科目名｜講師名｜教室名
| 微積分学101｜田中教授｜202号室
| 美術史｜山田（代理）｜ホールA
A. 以下のように、すべての科目名を抽出しました。

| 科目名 |
| --- |
| 微積分学101 |
| 美術史 |
| 物理学101 |
| 日本史 |
| 英語学習法 |
| 経済学101 |
| 倫理学 |
| 生物学101 |
| 地理学 |
| 哲学 |
| 心理学 |
| 社会学 |
| 音楽史 |
| 映画史 |
| 芸術史 |
| 建築史 |
| 考古学 |
| 文学史 |
| 言語学 |
| 精神科医学 |
| 薬学 |
| 病理学 |
| 組織学 |
| 細胞学 |
| 分子生物学 |
| 遺伝学 |
| 発生生物学 |
| 生態学 |
| 地質学 |
| 気象学 |
| 宇宙物理学 |
| 原子物理学 |
| 素粒子物理学 |
| 核物理学 |
| 応用物理学 |
| 医用生体工学 |
| 機械工学 |
| 電気工学 |
| 電子工学 |
| 情報工学 |
| 計算機科学 |
| 認識科学 |
| 人工知能 |
| ロボット工学 |
| 航空宇宙工学 |
| 土木工学 |
| 建築工学 |
| 交通工学 |
| 環境工学 |
| 海洋工学 |
| 土質工学 |
| 水質工学 |
| 経営学 |
| 経済学 |
| 統計学 |
| 法律学 |
| 政治学 |
| 国際関係論 |
| 国際政治経済学 |
| 社会学 |
| 教育学 |
| 心理学 |
| 臨床心理学 |
| カウンセリング学 |
| 職業訓練学 |
| 教育技術 |
| 教育心理学 |
| 学習心理学 |
| 特別支援教育 |
| 障がい者教育 |
| 芸術教育 |
| 音楽教育 |
| 美術教育 |
| 体育教育 |
| 英語教育 |
| 語学教育 |
| 翻訳学 |
| 通訳学 |
| 国際関係学 |
| 国際政治学 |
| 地域研究 |
| 文化研究 |
| 異文化コミュニケーション |
| グローバル・スタディーズ |
| 国際協力学 |
| 開発学 |
| 環境学 |
| 農学 |
| 森林科学 |
| 動物学 |
| 遺伝学 |
| 生物資源科学 |
| 生物技術 |
| 生物環境工学 |
| 生物医学

## Google Cloud Storage への保存

作成したデータセットは、Google Cloud Storage に保存する。

あらかじめ、Google Clod から `gs-colaboratory@japanese-llm-eval.iam.gserviceaccount.com` のJSONキーをダウンロードし ([リンク](https://console.cloud.google.com/iam-admin/serviceaccounts?project=japanese-llm-eval))、シークレット `GCP_KEY` として保存しておく。

In [ ]:
import os

from google.colab import userdata

model_name = "Xwin-LM/Xwin-LM-13B-V0.1"

service_account_info = userdata.get('GCP_KEY')

with open('gcp_service_account.json', 'w') as file:
    file.write(service_account_info)

!gcloud auth activate-service-account --key-file /content/gcp_service_account.json

!gsutil cp /content/preds.jsonl gs://japanese-llm-eval/{model_name}/preds.jsonl

Activated service account credentials for: [gs-colaboratory@japanese-llm-eval.iam.gserviceaccount.com]
Copying file:///content/preds.jsonl [Content-Type=application/octet-stream]...
/ [1 files][ 90.6 KiB/ 90.6 KiB]                                                
Operation completed over 1 objects/90.6 KiB.                                     
